Imports

In [1]:
from gensim.models import FastText

%run preprocessing.ipynb

15
Char [['m', 's', '>', 'l', 'p'], ['w', 'm', 'n'], ['H', 'n', 'v'], ['w', 'h', 'w'], ['q', 'A', 'd', 'r'], ['E', 'l', 'Y'], ['A', 'l', '<', 'T', 'E', 'A', 'm'], ['>', 'w'], ['A', 'l', 'k', 's', 'w', 'p'], ['>', 'w'], ['A', 'l', 'E', 't', 'q'], ['v', 'm'], ['A', 'f', 't', 'q', 'r'], ['f', 'E', 'j', 'z'], ['E', 'n'], ['k', 'l'], ['*', 'l', 'k'], ['l', 'm'], ['y', 'j', 'z', 'h'], ['A', 'l', 'S', 'w', 'm'], ['>', 'S', 'l', 'A']]
Diac [['a', 'o', 'a', 'a', 'N'], ['a', 'a', 'o'], ['a', 'i', 'a'], ['a', 'u', 'a'], ['a', ' ', 'i', 'N'], ['a', 'a', ' '], [' ', ' ', 'i', 'o', 'a', ' ', 'i'], ['a', 'o'], [' ', 'o', 'i', 'o', 'a', 'i'], ['a', 'o'], [' ', 'o', 'i', 'o', 'i'], ['u', '~a'], [' ', 'o', 'a', 'a', 'a'], ['a', 'a', 'a', 'a'], ['a', 'o'], ['u', '~i'], ['a', 'i', 'a'], ['a', 'o'], ['u', 'o', 'i', 'i'], [' ', ' ', '~a', 'o', 'u'], ['a', 'o', 'F', ' ']]


### Get unique characters and diacritics

In [2]:
unique_characters = ['A', 'b', 't', 'v', 'j', 'H', 'x', 'd', '*', 'r', 'z', 's', '$', 'S', 'D', 'T', 'Z', 'E', 'g', 'f', 'q', 'k', 'l', 'm', 'n', 'h', 'w', 'y', "'", '>', '<', '&', '}', '|', '{', '`', 'Y', 'p']
unique_diacritics = ['o', 'a', 'i', '~', 'u', 'N', 'F', 'K', ' ', '~a', '~i', '~u', '~N', '~F', '~K']

num_chars = len(unique_characters)
num_classes = len(unique_diacritics)

char_to_index = {char: i for i, char in enumerate(unique_characters)}
diacritic_to_index = {diacritic: i for i, diacritic in enumerate(unique_diacritics)}

print(num_chars)
print(num_classes)

38
15


Read File

In [3]:
def readFile(path):
	sentences = []
	with open(path, 'r', encoding='utf-8') as file:
		for line in file:
			sentences.append(line.strip())

	return sentences

In [4]:
TRAIN_PATH = "../dataset/train.txt"
VAL_PATH = "../dataset/val.txt"

Read Data 

In [5]:
def Read_data():
	corpus = readFile(TRAIN_PATH)
	loaded_model = FastText.load("./models/ft_model")
	X_train = []
	y_train = []
	embeddings_train = []
	max_sequence_length = 0

	for sentence in corpus:
		# Clean each sentence in the corpus
		clean_sentence = run_buckwalter(sentence.strip())
		# Get the char list for each word in the sentence and its corresponding diacritics
		char_list, diacritics_list = extract_labels(clean_sentence)

		X_train.append(char_list)
		y_train.append(diacritics_list)

		# Get the max sequence length and concatenate the embeddings of the words
		for word in char_list:
			max_sequence_length = max(max_sequence_length, len(word))

			embeddings_train.append(loaded_model.wv[word])

	# embeddings_train = np.concatenate(embeddings_train, axis=0)
	print(np.array(embeddings_train).shape)
	return X_train, y_train, embeddings_train, max_sequence_length




Padding Data

In [6]:
def padding_data( X_train, y_train,max_sequence_length):
	# Encoding and Padding the data
	X_train_padded = []
	for sentence in X_train:
		X_train_sequences = [[char_to_index[char] for char in word] for word in sentence]
		X_train_padded.append(pad_sequences(X_train_sequences, maxlen=max_sequence_length, padding='post'))

	# X_train_padded = np.concatenate(X_train_padded, axis=0)
	print(np.array(X_train_padded).shape)

	y_train_padded = []
	for sentence in y_train:
		y_train_sequences = [[diacritic_to_index[diacritic] for diacritic in diacritic_sequence] for diacritic_sequence in sentence]
		y_train_padded.append(pad_sequences(y_train_sequences, maxlen=max_sequence_length, padding='post'))

	# y_train_padded = np.concatenate(y_train_padded, axis=0)
	print(np.array(y_train_padded).shape)
	return X_train_padded, y_train_padded

Read Validation Data

In [7]:
def Read_val():
    val_corpus = readFile(VAL_PATH)
    loaded_model = FastText.load("./models/ft_model")
    X_val = []
    y_val = []
    embeddings_train = []
    for sentence in val_corpus:
        # Clean each sentence in the corpus
        clean_sentence = run_buckwalter(sentence.strip())
        # Get the char list for each word in the sentence and its corresponding diacritics
        char_list, diacritics_list = extract_labels(clean_sentence)

        X_val.append(char_list)
        y_val.append(diacritics_list)
        for word in char_list:

            embeddings_train.append(loaded_model.wv[word])
    return X_val, y_val,embeddings_train

Padding validation data

In [8]:
def padding_val(X_val, y_val, max_sequence_length):
    # Encoding and Padding the data
    X_val_padded = []
    for sentence in X_val:
        X_val_sequences = [[char_to_index[char] for char in word] for word in sentence]
        X_val_padded.append(pad_sequences(X_val_sequences, maxlen=max_sequence_length, padding='post'))

    # X_val_padded = np.concatenate(X_val_padded, axis=0)
    print(np.array(X_val_padded).shape)

    y_val_padded = []
    for sentence in y_val:
        y_val_sequences = [[diacritic_to_index[diacritic] for diacritic in diacritic_sequence] for diacritic_sequence in sentence]
        y_val_padded.append(pad_sequences(y_val_sequences, maxlen=max_sequence_length, padding='post'))

    # y_val_padded = np.concatenate(y_val_padded, axis=0)
    print(np.array(y_val_padded).shape)
    return X_val_padded, y_val_padded